In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
from itertools import permutations
import os

def fetch_intraday_forex_data(api_key, date, ticker, timespan="minute", multiplier=1, max_retries=3):
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{date}/{date}"
    
    params = {
        "adjusted": "true",
        "sort": "asc",
        "limit": 50000,
        "apiKey": api_key
    }

    retries = 0
    while retries <= max_retries:
        try:
            response = requests.get(url, params=params)
            if response.status_code == 429:
                print(f"Rate limit hit for {ticker} on {date}. Sleeping 60 seconds...")
                time.sleep(60)
                retries += 1
                continue
            elif response.status_code != 200:
                print(f"Error {response.status_code} for {ticker} on {date}: {response.text}")
                return None

            data = response.json()
            if 'results' not in data or not data['results']:
                print(f"No results for {ticker} on {date}")
                return None

            return pd.DataFrame([{
                "date": date,
                "pair": ticker[2:],
                "timestamp": pd.to_datetime(r['t'], unit='ms'),
                "open": r.get("o", 0),
                "high": r.get("h", 0),
                "low": r.get("l", 0),
                "close": r.get("c", 0),
                "volume": r.get("v", 0)
            } for r in data["results"]])

        except requests.exceptions.RequestException as e:
            print(f"Network error for {ticker} on {date}: {e}")
            time.sleep(60)
            retries += 1

    print(f"Failed to fetch data for {ticker} on {date} after {max_retries} retries.")
    return None

API_KEY = "vSmtvpdrv8xGtZG7YYgSiPg3irGrpWbd"
currencies = ["USD", "EUR", "JPY", "GBP", "CNH", "AUD", "CAD", "CHF", "HKD", "SGD"]
currency_pairs = [f"C:{a}{b}" for a, b in permutations(currencies, 2)]

start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 3, 30)

output_folder = "Data"
os.makedirs(output_folder, exist_ok=True)

current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime('%Y-%m-%d')
    filename = os.path.join(output_folder, f"forex_intraday_{date_str}.csv")

    if os.path.exists(filename):
        print(f"File already exists for {date_str}, skipping...")
        current_date += timedelta(days=1)
        continue

    all_data = []
    print(f"\nFetching data for {date_str}...")

    for i, pair in enumerate(currency_pairs):
        print(f"[{i+1}/{len(currency_pairs)}] → {pair}")
        df = fetch_intraday_forex_data(API_KEY, date_str, pair)
        if df is not None:
            all_data.append(df)
        time.sleep(1.2)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        filename = os.path.join(output_folder, f"forex_intraday_{date_str}.csv")
        final_df.to_csv(filename, index=False)
        print(f"Saved {len(final_df)} rows to {filename}")
    else:
        print("No data fetched for this day.")

    current_date += timedelta(days=1)


📅 Fetching data for 2025-01-01...
[1/90] → C:USDEUR
[2/90] → C:USDJPY
[3/90] → C:USDGBP
[4/90] → C:USDCNH
[5/90] → C:USDAUD
[6/90] → C:USDCAD
⚠️ Rate limit hit for C:USDCAD on 2025-01-01. Sleeping 60 seconds...
[7/90] → C:USDCHF
[8/90] → C:USDHKD
[9/90] → C:USDSGD
[10/90] → C:EURUSD
[11/90] → C:EURJPY
[12/90] → C:EURGBP
⚠️ Rate limit hit for C:EURGBP on 2025-01-01. Sleeping 60 seconds...
[13/90] → C:EURCNH
[14/90] → C:EURAUD
[15/90] → C:EURCAD
[16/90] → C:EURCHF
[17/90] → C:EURHKD
⚠️ Rate limit hit for C:EURHKD on 2025-01-01. Sleeping 60 seconds...
[18/90] → C:EURSGD
[19/90] → C:JPYUSD
[20/90] → C:JPYEUR
[21/90] → C:JPYGBP
[22/90] → C:JPYCNH
⚠️ Rate limit hit for C:JPYCNH on 2025-01-01. Sleeping 60 seconds...
⚠️ No results for C:JPYCNH on 2025-01-01
[23/90] → C:JPYAUD
[24/90] → C:JPYCAD
[25/90] → C:JPYCHF
[26/90] → C:JPYHKD
[27/90] → C:JPYSGD
⚠️ Rate limit hit for C:JPYSGD on 2025-01-01. Sleeping 60 seconds...
⚠️ No results for C:JPYSGD on 2025-01-01
[28/90] → C:GBPUSD
[29/90] → C:GBP